In [1]:
!pip install scikit-learn


In [3]:
# Make necessary imports.
import warnings
warnings.filterwarnings("ignore")

import numpy as np 
import pandas as pd
import os, sys
import sklearn
from sklearn.preprocessing import MinMaxScaler
# from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

ImportError: cannot import name 'plot_precision_recall_curve' from partially initialized module 'sklearn.metrics' (most likely due to a circular import) (C:\Users\Vedant\anaconda3\lib\site-packages\sklearn\metrics\__init__.py)

# PREPROCCESING

In [ ]:
#load the dataset:-
df = pd.read_csv(r'C:\Users\Vedant\Desktop\PBL\kidney_disease.csv')
df.head(n=10)

In [ ]:
df.shape  #to check whether teh data set loaded properly or not

In [ ]:
#Find the null values in the dataset
df.isnull().sum()  #gives how many null values per columns

In [ ]:
#imputing null values

from sklearn.impute import SimpleImputer
imp_mode = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')

df_imputed = pd.DataFrame(imp_mode.fit_transform(df))
df_imputed.columns = df.columns
df_imputed

In [ ]:
#we can see now there is no null values
df_imputed.isnull().sum() 

In [ ]:
df_imputed.columns   #Gives the list of columns.

In [ ]:
set(df_imputed["age"].tolist())   #tolist- to convert into list, and set()- to keep only unique values in the list.

In [ ]:
#find the unique values in every column.

for i in df_imputed.columns:
    print("*****************************************************",i,"*********************************************************")
    print()
    print(set(df_imputed[i].tolist()))
    print()

In [ ]:
#In dataset we found that there are many mistaken values, so we replacing mistaken values with actual values.

print(df_imputed["rc"].mode())
print(df_imputed["wc"].mode())
print(df_imputed["pcv"].mode())

In [ ]:
#here we using if/else inline function to replace values.

df_imputed["classification"]=df_imputed["classification"].apply(lambda x:'ckd' if x=="ckd\t" else x)

df_imputed["cad"]=df_imputed["cad"].apply(lambda x:'no' if x=="\tno" else x)

df_imputed["dm"]=df_imputed["dm"].apply(lambda x:'no' if x=="\tno" else x)
df_imputed["dm"]=df_imputed["dm"].apply(lambda x:'yes' if x=="\tyes" else x)
df_imputed["dm"]=df_imputed["dm"].apply(lambda x:'yes' if x=="yes" else x)

df_imputed["rc"]=df_imputed["rc"].apply(lambda x:'5.2' if x=="\t?" else x)

df_imputed["wc"]=df_imputed["wc"].apply(lambda x:'9800' if x=="\t6200" else x)
df_imputed["wc"]=df_imputed["wc"].apply(lambda x:'9800' if x=="\t8400" else x)
df_imputed["wc"]=df_imputed["wc"].apply(lambda x:'9800' if x=="\t?" else x)

df_imputed["pcv"]=df_imputed["pcv"].apply(lambda x:'41' if x=="\t43" else x)
df_imputed["pcv"]=df_imputed["pcv"].apply(lambda x:'41' if x=="\t?" else x)

In [ ]:
#We can see that all the errors are been removed.

for i in df_imputed.columns:
    print("*****************************************************",i,"*********************************************************")
    print()
    print(set(df_imputed[i].tolist()))
    print()

In [ ]:
temp = df_imputed["classification"].value_counts()
temp

In [ ]:
#Check Label Imbalance:- 
#label imbalance -uneven distribution of classes in a classification problem, where one or more classes have significantly fewer instances compared to others.


temp = df_imputed["classification"].value_counts()
temp_df = pd.DataFrame({'classification': temp.index, 'values': temp.values})
print(sns.barplot(x= 'classification', y ="values", data = temp_df))

In [ ]:
df.dtypes

In [ ]:
#Fixing datatypes

df_imputed.dtypes

In [ ]:
#Excluding datatype 'object'

df.select_dtypes(exclude = ["object"]).columns

In [ ]:
#Changing datatypes 'object' to 'float'.

for i in df.select_dtypes(exclude = ["object"]).columns:
    df_imputed[i] = df_imputed[i].apply(lambda x: float(x)) 

In [ ]:
df_imputed.dtypes

In [ ]:
#EDA and Visualizations:-

In [ ]:
sns.pairplot(df_imputed)    #Plots the relation b/w the pair of the attributes.

# scatters - numerical columns
# straight lines - categorical columns

In [ ]:
#Find the distribution of data.

def distplots(col):
    sns.distplot(df[col])
    plt.show()
    
for i in (df.select_dtypes(exclude = ["object"]).columns)[1:]:
    distplots(i)

In [ ]:
#Finding and removing the outliers:-
#Outliers- Outliers are data points that are significantly different from the majority of the data.
#They can be unusually high or low values compared to the rest of the dataset. Outliers can occur due to errors, rare events,
#or natural variations. 

def boxplots(col):
    sns.boxplot(df[col])
    plt.show()
    
for i in (df.select_dtypes(exclude = ["object"]).columns)[1:]:
    boxplots(i)

In [ ]:
#FINAL PART OF PREPROCCESING:-
#Encoding the values,that means there will be no 'yes' or 'no' in the data set. ALl the values are replace by '0' and '1' and 
# Numerical or integer values remains same.

from sklearn import preprocessing

df_enco = df_imputed.apply(preprocessing.LabelEncoder().fit_transform)
df_enco

#As we can see, all the values are get replaced.

In [ ]:
#HERE WE COMPLETED ENITRE PREPROCESSING PART.

In [ ]:
df_enco.to_csv("kidney_disease_pre-processed.csv")

In [ ]:
#Finding Correlations

plt.figure(figsize=(20,20))
corr = df_enco.corr()
sns.heatmap(corr,annot = True)

In [ ]:
#Lets make the final changes to the data.
#Seperate the independent data(x) and dependent data(y)
#dependent data/attribute in 'classification' which is dependent on other attributes. And data other than 'classification'
#independent data.

#Now, dropping 'id' columns from data bcoz we don't have need.

x = df_enco.drop(["id", "classification"], axis=1)
y = df_enco["classification"]

In [ ]:
#Lets detect the label balance
import collections
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
print(Counter(y))

In [ ]:
#lets balance the label 

ros = RandomOverSampler()
x_ros, y_ros = ros.fit_resample(x,y)
print(Counter(y_ros))

#As we can see, labels are balanced now.
#means ckd = 250 , notckd = 250

In [ ]:
#Initialize the MinMaxScaler and scale the features b/w -1 and 1 to normalize them.
#The MinMaxScaler transform the features by scaling them to a given range.
#The fit_transform() method fits to the data and then transform it. We dont need to scale the labels
#Scale the features to b/w -1 and 1.

#Scaling is important in the algorithm such as support vector machine(SVM) and k-nearest neighbout(KNN) where the distance b/w 
#the data point is important.

scaler = MinMaxScaler((-1,1))
x = scaler.fit_transform(x_ros)
y = y_ros

In [ ]:
#Dimensionality reduction
#Applying PCA

#The code below is .95 for the number of component parameter.
#It means that Skcit-learn choose the minimum number of principal components such that 95% of variance is retained.

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from sklearn.decomposition import PCA

pca = PCA(.95)
X_PCA = pca.fit_transform(x)

print(x.shape)
print(X_PCA.shape)
#Thus we need 18 columns to keep 95% of the variance.

In [ ]:
#With PCA 
#Now split the dataset into training and testing sets keeping 20% of the data for the testing.
#Split the dataset.
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_PCA, y,test_size=0.2, random_state=7)
#A sequential model is appropriate for a plain stack of layers where each layer has exactly one inout and one ouput  and one output is connected to the another input
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout


# from tensorflow.keras.callbacks import ModelCheckpoints, EarlyStopping
# from keras.models import Sequential, Model

# from keras.optimizers import Adam
# from sklearn.model_selection import KFold

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

from keras.models import Sequential, Model

from keras.optimizers import Adam
from sklearn.model_selection import KFold

In [ ]:
# from keras.models import Sequential, Model

# from keras.optimizers import Adam
# from sklearn.model_selection import KFold

In [ ]:
def model():
    
    classifier = Sequential()
    classifier.add(Dense(15, input_shape = (x_train.shape[1],), activation = 'relu'))
    classifier.add(Dropout(0.2))
    classifier.add(Dense(15, activation = 'relu'))
    classifier.add(Dropout(0.4))
    classifier.add(Dense(1, activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    
    return classifier


In [ ]:
model = model()     #To be check 
model.summary()



In [ ]:
history = model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = 5, verbose = 1)

In [ ]:
# import scikitplot
from sklearn.metrics import roc_curve, auc, confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import precision_recall_curve, average_precision_score, plot_precision_recall_curve, f1_score,confusion_matrix

In [ ]:
#Function to plot the validation curve.

def plot_auc(t_y, p_y):
    fpr, tpr, thresholds = roc_curve(t_y, p_y, pos_label = 1)
    fig, c_ax = plt.subplots(1,1,figsize = (9,9))
    c_ax.plot(fpr, tpr, label = '%s(AUC:0.2f)' % ('classification', auc(fpr,tpr)))
    c_ax.plot([0,1], [0,1], color = 'navy', lw = 1, linestyle = '==')
    c_ax.legend()
    c_ax.set_xlabel('false positive rate')
    c_ax.set_ylabel('true positive rate')
    
#function to plot the precision recall curve.You can utilize at precision_recall_curve imported above.
def plot_precision_recall_helper(t_y, p_y):
    fig, c_ax = plt.subplots(1,1,figsize = (9,9))
    precision, recall, thresholds = precision_recall_curve(t_y, p_y, pos_label = 1)
    aps = averge_precision_score(t_y, p_y)
    c_ax.plot(recall,precision, label = '%s(AUC:0.2f)' % ('classification', aps))
    c_ax.plot(recall, precision, color = 'red', lw = 1)
    c_ax.legend()
    c_ax.set_xlabel('recall')
    c_ax.set_ylabel('precision')
    
#function to plot the history
def plot_history(history):
    f = plt.figure()
    f.set_figwidth(15)
    
    f.add_subplot(1,2,1)
    plt.plot(history.history['val_loss'], label = 'val loss')
    plt.plot(history.history['loss'], label = 'train loss')
    plt.legend()
    plt.title('Model loss')
    
    f.add_subplot(1,2,2)
    plt.plot(history.history['val_accuracy'], label = 'val accuracy')
    plt.plot(history.history['accuracy'], label = 'train accuracy')
    plt.legend()
    plt.title('Model Accuracy')
    
    plt.show()

In [ ]:
hist = plot_history(history)

In [ ]:
plot_auc(y_test, model.predict(x_test, verbose = True))